## 结合工具规划逻辑

包括 ReAct、ReWoo、Plan 等复杂的工具规划推理也可以通过 FlowAgent 实现。

## ReAct

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReAct

def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReAct(planner=ChatQwen(), tools=[get_city, get_weather, booking])
flow("我在鸿蒙公司，帮我查询一下天气情况")

flow.agents[0].memory

[AGENT] STEP 1 >>> Node 1: ChatQwen.4380704432
**思考**  
首先，我需要确定鸿蒙公司的具体位置或所在城市，因为这将帮助我查询该地区的天气情况。接下来，我会使用 `get_city` 工具来获取鸿蒙公司所在的城市信息。

**行动**  
Plan: 查询鸿蒙公司所在的详细城市. #E1 = get_city[{"location": "鸿蒙公司"}]
广州

**观察** 上面的行动结果为:
广州
[AGENT] STEP 2 >>> Node 1: ChatQwen.4380704432
**思考**  
既然已经知道鸿蒙公司位于广州，那么下一步就是查询广州的天气情况。为了获得这个信息，我将使用 `get_weather` 工具。

**行动**  
Plan: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]
广州
广州天气不错，一直是大晴天

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天
[AGENT] STEP 3 >>> Node 1: ChatQwen.4380704432
**思考**  
我已经获得了鸿蒙公司所在的城市（广州）以及广州的天气情况（天气不错，一直是大晴天）。基于这些信息，我认为没有必要采取进一步的行动，可以直接提供最终的答案。

**最终答案**  
鸿蒙公司所在的广州天气非常好，一直保持晴朗。希望这个信息对你有帮助！


[{'role': 'system',
  'content': '尽你所能完成任务。\n\n**你要解决的问题是: ** 我在鸿蒙公司，帮我查询一下天气情况\n\n\n**思考**  \n首先，我需要确定鸿蒙公司的具体位置或所在城市，因为这将帮助我查询该地区的天气情况。接下来，我会使用 `get_city` 工具来获取鸿蒙公司所在的城市信息。\n\n**行动**  \nPlan: 查询鸿蒙公司所在的详细城市. #E1 = get_city[{"location": "鸿蒙公司"}]\n\n**观察** 上面的行动结果为:\n广州\n\n**思考**  \n既然已经知道鸿蒙公司位于广州，那么下一步就是查询广州的天气情况。为了获得这个信息，我将使用 `get_weather` 工具。\n\n**行动**  \nPlan: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]\n\n**观察** 上面的行动结果为:\n广州\n广州天气不错，一直是大晴天\n\n\n(现在不要急于解决问题，而是继续按照如下步骤一步一步输出你的推理过程。)\n\n**思考** \n对当前情况进行反思, 然后说明你现在的决策：当前就结束任务并输出**最终答案**，还是继续考虑下一步行动的应当如何执行。\n\n**最终答案**\n(在此输出你的最终答案。)\n如果输出最终答案，就结束任务，停止所有输出。\n\n**行动** \n你可以尝试直接解决问题，并输出解决的详细过程；\n或者从 ** get_city,get_weather,booking ** 中选择一个工具来解决问题。\n\n这些工具的详细描述如下:\n{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}\n{"type": "function", "function": {"name": "get_weather", "des

In [2]:
print(flow.agents[0].memory[0]['content'])

尽你所能完成任务。

**你要解决的问题是: ** 我在鸿蒙公司，帮我查询一下天气情况


**思考**  
首先，我需要确定鸿蒙公司的具体位置或所在城市，因为这将帮助我查询该地区的天气情况。接下来，我会使用 `get_city` 工具来获取鸿蒙公司所在的城市信息。

**行动**  
Plan: 查询鸿蒙公司所在的详细城市. #E1 = get_city[{"location": "鸿蒙公司"}]

**观察** 上面的行动结果为:
广州

**思考**  
既然已经知道鸿蒙公司位于广州，那么下一步就是查询广州的天气情况。为了获得这个信息，我将使用 `get_weather` 工具。

**行动**  
Plan: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天


(现在不要急于解决问题，而是继续按照如下步骤一步一步输出你的推理过程。)

**思考** 
对当前情况进行反思, 然后说明你现在的决策：当前就结束任务并输出**最终答案**，还是继续考虑下一步行动的应当如何执行。

**最终答案**
(在此输出你的最终答案。)
如果输出最终答案，就结束任务，停止所有输出。

**行动** 
你可以尝试直接解决问题，并输出解决的详细过程；
或者从 ** get_city,get_weather,booking ** 中选择一个工具来解决问题。

这些工具的详细描述如下:
{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}
{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "

In [3]:
print(flow.provider_dict['completed_work'])

**思考**  
首先，我需要确定鸿蒙公司的具体位置或所在城市，因为这将帮助我查询该地区的天气情况。接下来，我会使用 `get_city` 工具来获取鸿蒙公司所在的城市信息。

**行动**  
Plan: 查询鸿蒙公司所在的详细城市. #E1 = get_city[{"location": "鸿蒙公司"}]

**观察** 上面的行动结果为:
广州

**思考**  
既然已经知道鸿蒙公司位于广州，那么下一步就是查询广州的天气情况。为了获得这个信息，我将使用 `get_weather` 工具。

**行动**  
Plan: 查询广州的天气情况. #E2 = get_weather[{"city": "广州"}]

**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天

**思考**  
我已经获得了鸿蒙公司所在的城市（广州）以及广州的天气情况（天气不错，一直是大晴天）。基于这些信息，我认为没有必要采取进一步的行动，可以直接提供最终的答案。

**最终答案**  
鸿蒙公司所在的广州天气非常好，一直保持晴朗。希望这个信息对你有帮助！


In [4]:
flow.final_answer

'鸿蒙公司所在的广州天气非常好，一直保持晴朗。希望这个信息对你有帮助！'

## ReWOO

- 管道计算场景：上一个任务的输出，作为下一个任务的输入
- 参数组装场景：上一个任务的输出需要做一定转换，才能作为下一个任务的输入（执行哪个任务是确定的，但参数需要组装）
- 条件分支场景：上一个任务的输出作为一种判定条件，决定执行接下来哪个任务（执行哪个任务不确定，需要根据当前任务返回来判定）

In [1]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen, ReWOO
def get_city(location: str):
    """由任意地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

flow = ReWOO(planner=ChatQwen(), tools=[get_city, get_weather, booking])
flow("我在鸿蒙公司，帮我查询一下天气情况")
flow.agents[0].memory

[AGENT] STEP 1 >>> Node 1: planner
Plan: 首先，需要确定鸿蒙公司的具体位置或所在城市。
#E1 = get_city({"location": "鸿蒙公司"})

Plan: 根据获取到的城市信息，查询该城市的天气情况。
#E2 = get_weather({"city": "#E1"})
广州


**观察** 上面的行动结果为:
广州
广州天气不错，一直是大晴天


**观察** 上面的行动结果为:
广州天气不错，一直是大晴天


[{'role': 'system',
  'content': 'For the following task, make plans that can solve the problem step by step. \nFor each plan, indicate which external tool together with tool input to retrieve evidence. \nYou can store the evidence into a variable #E that can be called by later tools. \n(Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n{"type": "function", "function": {"name": "get_city", "description": "由任意地址描述查询出所在的城市", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": ""}}, "required": ["location"]}}}\n{"type": "function", "function": {"name": "get_weather", "description": "我可以查询城市的天气情况。city必须是城市名称。", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": ""}}, "required": ["city"]}}}\n{"type": "function", "function": {"name": "booking", "description": "我可以根据你的需求预订球场", "parameters": {"type": "object", "properties": {"request": {"type": "string", "description": ""}}, "required": ["re

In [2]:
flow.handler_tool_call.steps

[{'id': '#E1',
  'description': '首先，需要确定鸿蒙公司的具体位置或所在城市。',
  'name': 'get_city',
  'arguments': '{"location": "鸿蒙公司"}',
  'result': '广州'},
 {'id': '#E2',
  'description': '根据获取到的城市信息，查询该城市的天气情况。',
  'name': 'get_weather',
  'arguments': '{"city": "#E1"}',
  'result': '广州天气不错，一直是大晴天'}]